# Coursera_Capstone


In [2]:
#import packages that may e needed
import pandas as pd
import numpy as np

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

import matplotlib.pyplot as plt
%matplotlib inline 

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



PART 1, DATA CLEANSING

In [3]:
#define a variable url to store htm link there 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#import data from html and pass url as an argument. As read produces a list of a data frames we further need to access desired one via index [0] 
data = pd.read_html(url)[0]
#give names to columns
data.columns = ['Postcode','Borough','Neighbourhood']
#ignore 'not assigned'
data = data[data.Borough != 'Not assigned']
#condition on the repeating, use groupby to gtoup y postcode and use aggregatefunction to gather values together & reset index from to get a nice looking table with 3 columns
data = data.groupby("Postcode").agg(lambda x:','.join(set(x))).reset_index()
#Now,if cell has a borough but neighborhood is na, then the neighborhood == borough
#Loop through the data frame
for i, j in data.iterrows():
    if j['Neighbourhood']== 'Not Assigned' and j['Borough']!= 'Not Assigned':
        j['Neighbourhood']= j['Borough']
data       

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


PART 2 - DF with coordinates

In [4]:
# create variable url_2 to store link to csv
url_2='http://cocl.us/Geospatial_data'
#read the file passing url_2 as an argument
coord = pd.read_csv(url_2)
#give colums names
coord.columns = ['Postcode','Latitude','Longitude']
#merge tables via postcode as it is the only overlapping column, we get the table needed but in a shuffled order
data = pd.merge(coord, data, on='Postcode').reset_index()
#let us change the place of each column
data = data[['Postcode','Borough', 'Neighbourhood', 'Latitude','Longitude']]
data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
